In [1]:
import sys
sys.path.insert(0, '..')

from functools import partial
import pandas as pd
import intransparent as it

it.show(f'<h1>CSAM Reports per Country</h1>')
it.show(f'<h2>Data Schemas</h2>')

logger = partial(it.show, show_schema=True, margin_bottom=2)
country_data = it.ingest_reports_per_country('../data', logger=logger)


index,iso3,:,category,(no nulls)
index,year,:,category,(no nulls)
column,reports,:,int64,(no nulls)
column,reports_pct,:,float64,(no nulls)


index,iso3,:,category,(no nulls)
index,year,:,category,(no nulls)
column,population,:,int64,(no nulls)
column,population_pct,:,float64,(no nulls)


index,iso3,:,category,(no nulls)
index,year,:,category,(no nulls)
column,online,:,int64,(no nulls)
column,online_pct,:,float64,(no nulls)


index,iso3,:,category,(no nulls)
column,country,:,category,(no nulls)
column,iso2,:,category,(no nulls)
column,region,:,category,(no nulls)


index,,:,int64,(no nulls)
column,superregion,:,category,(no nulls)
column,region,:,category,(no nulls)
column,continent,:,category,(no nulls)


index,,:,int64,(no nulls)
column,iso3,:,category,(no nulls)


index,iso3,:,category,(no nulls)
index,year,:,category,(no nulls)
column,reports,:,int64,(no nulls)
column,reports_pct,:,float64,(no nulls)
column,reports_per_capita,:,float64,(no nulls)
column,population,:,int64,(no nulls)
column,population_pct,:,float64,(no nulls)
column,online,:,float64,(872 nulls)
column,online_pct,:,float64,(872 nulls)
column,country,:,category,(no nulls)
column,iso2,:,category,(no nulls)


In [2]:
it.show(f'<h2>Missing Population Data</h2>')
countries_without, reports_without = it.without_populations(
    country_data.reports, country_data.populations, country_data.countries
)
it.show(
    f'{countries_without.shape[0]} countries have reports but not population '
    'statistics. They also account for very few reports:'
)
it.show(
    reports_without,
    caption='Reports for Countries<br>w/o Population Stats',
)
it.show('The countries are:')
it.show(countries_without, caption='Countries without Population Data in UN Dataset')


,reports,reports_pct
year,,
2019,28,0.000165
2020,97,0.000446
2021,243,0.000827
2022,117,0.000365
2023,58,0.000160


,country,region
iso3,,
ALA,Åland Islands,Northern Europe
ANT,Netherlands Antilles,Caribbean
ATA,Antarctica,Antarctica
ATF,French Southern Territories,Eastern Africa
BVT,Bouvet Island,South America
CCK,Cocos (Keeling) Islands,Australia and New Zealand
CXR,Christmas Island,Australia and New Zealand
EEE,Europe,Europe
IOT,British Indian Ocean Territory,Eastern Africa


In [3]:
df = (
    country_data.reports
    .reset_index()
    .pipe(lambda df: df[df['iso3'].isin(countries_without.index) & (df['reports'] > 0)])
    .merge(country_data.countries, how='left', on='iso3')
    [['iso3', 'country', 'year', 'reports', 'reports_pct']]
)
it.show(df, caption='Reports for Countries w/o Population Data', with_index=False)

iso3,country,year,reports,reports_pct
ALA,Åland Islands,2019,15,0.00008830
ALA,Åland Islands,2020,20,0.00009195
ALA,Åland Islands,2021,21,0.00007143
ALA,Åland Islands,2022,51,0.00015908
ALA,Åland Islands,2023,25,0.00006904
ANT,Netherlands Antilles,2019,1,0.00000589
ANT,Netherlands Antilles,2020,10,0.00004597
ANT,Netherlands Antilles,2021,2,0.00000680
ANT,Netherlands Antilles,2022,1,0.00000312
ATA,Antarctica,2019,1,0.00000589
